In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

#Loading functions from other notebooks
from ipynb.fs.full.race_timestamps import start_and_end_of_start
from ipynb.fs.full.sequence import start_sequence

# Defining leaderboard crossing the last mark
>It is interesting to know the position of the boats when they are rounding the first mark - the reach mark. This position is defining whether you had a good start or not. 
>It will be defined from looking at the number of leg that the boats are at, meaning that from start to reach mark is leg 1 and then as soon as the boat crosses the reach mark, leg 2 starts. Whoever boat is on leg 2 first are leading around the reach mark. 

In [8]:
 
def position_at_last_mark(day,sequence):
    
    leaderboard = pd.DataFrame()
    for start,end in sequence:
        race = (day[(day['DATETIME']>=start) & (day['DATETIME']<=end)]).reset_index(drop = True)
        
        #Defining the list of boats that has pasted the mark
        L2 = []        
        m = []
        unq_race = []
        #Create a list with the ranking of the boats
        ranking = []
        #The position that the boat has rounded the mark
        position1 = 0
        position2 = 0
        position3 = 0
        position4 = 0
        position5 = 0
        position6 = 0
        #Crossing starting line - leg 1
        for i in range(0,len(race)):
            if list(race['TRK_LEG_NUM_unk'])[i] == 1:
                if list(race['BOAT'])[i] not in L2: 
                    m.append(1.0)
                    L2.append(list(race['BOAT'])[i])
#                     print(L2)
                    position1 =position1+1
                    ranking.append(position1)
                    unq_race.append(race['TRK_RACE_NUM_unk'][i])
        #Crossing reach mark -  leg 2
            if list(race['TRK_LEG_NUM_unk'])[i] == 2:
                if L2.count(list(race['BOAT'])[i]) == 1: 
#                     print(L2)
                    m.append(2.0)
                    L2.append(list(race['BOAT'])[i])
                    position2 =position2+1
                    ranking.append(position2)
                    unq_race.append(race['TRK_RACE_NUM_unk'][i])
        #Crossing first ww mark - leg 3
            if list(race['TRK_LEG_NUM_unk'])[i] == 3:
                if L2.count(list(race['BOAT'])[i]) == 2: 
                    m.append(3.0)
                    L2.append(list(race['BOAT'])[i])
                    position3 =position3+1
                    ranking.append(position3)
                    unq_race.append(race['TRK_RACE_NUM_unk'][i])
        #Crossing first lw mark - leg 4
            if list(race['TRK_LEG_NUM_unk'])[i] == 4:
                if L2.count(list(race['BOAT'])[i]) == 3: 
                    m.append(4.0)
                    L2.append(list(race['BOAT'])[i])
                    position4 =position4+1
                    ranking.append(position4)
                    unq_race.append(race['TRK_RACE_NUM_unk'][i])
        #Crossing second ww mark - leg 5
            if list(race['TRK_LEG_NUM_unk'])[i] == 5:
                if L2.count(list(race['BOAT'])[i]) == 4: 
                    m.append(5.0)
                    L2.append(list(race['BOAT'])[i])
                    position5 =position5+1
                    ranking.append(position5)
                    unq_race.append(race['TRK_RACE_NUM_unk'][i])
        #Crossing second ww mark - leg 6
            if list(race['TRK_LEG_NUM_unk'])[i] == 6:
                if L2.count(list(race['BOAT'])[i]) == 5: 
                    m.append(6.0)
                    L2.append(list(race['BOAT'])[i])
                    position6 =position6+1
                    ranking.append(position6)
                    unq_race.append(race['TRK_RACE_NUM_unk'][i])

            leaderboard_midlertidig = pd.DataFrame(np.column_stack([unq_race, L2, m, ranking]),columns=['TRK_RACE_NUM_unk','BOAT', 'TRK_LEG_NUM_unk', 'POSITION_AT_LAST_MARK*'])
## Det var ét ræs- altså midlertidigt. Nu går vi en ud
        leaderboard=leaderboard.append(leaderboard_midlertidig,ignore_index=True)

 
    #First we need to create a unique column:
    leaderboard['joincolumn'] = leaderboard.apply(lambda row: row.TRK_RACE_NUM_unk + row.BOAT + row.TRK_LEG_NUM_unk , axis=1)

    #Change float64 to string to be able to add the columns for the unique column
    
#     df = start_sequence(day)
    
    day['TRK_RACE_NUM_unk'] = day['TRK_RACE_NUM_unk'].astype(str)
    day['TRK_LEG_NUM_unk'] = day['TRK_LEG_NUM_unk'].astype(str)
    day['joincolumn'] = day.apply(lambda row: row.TRK_RACE_NUM_unk + row.BOAT + row.TRK_LEG_NUM_unk , axis=1)

    #Now merging df and leaderboard
    df1 = pd.merge(day, leaderboard, on='joincolumn', how='left')
    df_starter = df1.drop(['TRK_LEG_NUM_unk_y','TRK_RACE_NUM_unk_y','BOAT_y','joincolumn'], axis=1)
    df_starter.rename(columns={'TRK_LEG_NUM_unk_x': 'TRK_LEG_NUM_unk', 'TRK_RACE_NUM_unk_x': 'TRK_RACE_NUM_unk','BOAT_x':'BOAT'}, inplace=True)
    #Prepair the data
    df_starter = df_starter.sort_values(by="DATETIME").reset_index(drop=True)
    df_starter["TRK_LEG_NUM_unk"]=df_starter["TRK_LEG_NUM_unk"].astype(float)
    df_starter["TRK_RACE_NUM_unk"]=df_starter["TRK_RACE_NUM_unk"].astype(float)
    df_starter["POSITION_AT_LAST_MARK*"]=df_starter["POSITION_AT_LAST_MARK*"].astype(float)
        

    return(df_starter)



In [9]:
# df =position_at_last_mark(df_race,start_times)